# DealInfos 를 ES에 벌크 인서트

-  환경: ElasticSearch 6
-  테스트 머신: twiceSpark1

```
PUT /dealinfos
{
mapping_dealinfos.json
}
```


In [1]:
import requests
import time
import urllib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import json
import elasticsearch
import csv
import pickle
from elasticsearch.helpers import bulk
import re
import glob
import os
from datetime import timezone, timedelta, datetime

In [2]:
def make_documents(file_path):
    pred_dic = {}
    with open(file_path, 'rb') as jsonfile:
        for row in jsonfile:
            source = eval(row)
            source['sst'] = datetime.fromtimestamp(source['sst'])
            source['set'] = datetime.fromtimestamp(source['set'])
            source['dc'] = source['dc'].replace(" ", "") # 편의상 공백을 모두 지워버림
            doc =  {
                "_op_type":'create',
                "_index":"dealinfos",
                "_type":"doc",
                "_id": source['did'],
                "_source": source
            }
            yield(doc)

In [3]:
### Jangmin's PC
es_url = 'localhost:9200'

In [4]:
es = elasticsearch5.Elasticsearch(es_url)

In [5]:
es.indices.get_settings(index='dealinfos')

{'dealinfos': {'settings': {'index': {'analysis': {'analyzer': {'my_analyzer': {'filter': ['my_length_filter',
        'lowercase'],
       'tokenizer': 'my_tokenizer',
       'type': 'custom'},
      'my_analyzer_comma': {'filter': ['my_length_filter', 'lowercase'],
       'tokenizer': 'my_tokenizer_comma_only',
       'type': 'custom'}},
     'filter': {'my_length_filter': {'max': '20',
       'min': '2',
       'type': 'length'}},
     'tokenizer': {'my_tokenizer': {'pattern': '[,/]',
       'type': 'simple_pattern_split'},
      'my_tokenizer_comma_only': {'pattern': '[,]',
       'type': 'simple_pattern_split'}}},
    'creation_date': '1528198539473',
    'number_of_replicas': '1',
    'number_of_shards': '5',
    'provided_name': 'dealinfos',
    'uuid': '-YoGsxyUSkme8njbHu_Oew',
    'version': {'created': '6020499'}}}}}

# bulk inserting to ES

In [ ]:
#data_dir = r'd:\WMIND\temp\dealinfo_small'
data_dir = r'd:\WMIND\temp\dealinfo'
start = time.time()
total_time = 0
bulk(es, make_documents(data_dir))
total_time += (time.time() - start)
print("Elapsed Time {} sec".format(total_time))

In [27]:
res = es.search(index='dealinfos', body={'query': {'match':{'dc':'에버랜드'}}})

In [28]:
res['hits']['total']

832